In [1]:
from webscraping import import_marques_modeles, fusionner_fichiers_json
from datacleaning import *
import polars as pl

In [2]:
FICHIERS_JSON = ["data_a_d.json", "data_e_l.json", "data_m_p.json", "data_r_z.json"]

nom_marques_modeles = import_marques_modeles()
data_brutes = fusionner_fichiers_json(FICHIERS_JSON)

In [3]:
data_brutes = pl.DataFrame(data_brutes)
nom_marques_modeles = pl.DataFrame(nom_marques_modeles)

In [4]:
data_brutes.shape

(308284, 10)

In [6]:
data = gazoduc(data_brutes, nom_marques_modeles)

In [7]:
data.shape

(273081, 16)

In [8]:
data.null_count()

annee,kilometrage,boite,energie,prix,position_marché,garantie,lien,marque,modele,generation,cylindre,moteur,puissance,finition,batterie
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,15052,53431,0,4668,258021


In [ ]:
data.write_parquet('data/database.parquet')

In [6]:
import duckdb

In [19]:
data = duckdb.sql(
    """
SELECT *
FROM 'data/database.parquet'
    """
).pl()

# test imputation cylindre + puissance

```python
from pandas import DataFrame as pandasDataFrame
class ImputerCylindrePuissance:
    def __init__(self, data: pandasDataFrame, target_features=['cylindre', 'puissance'], categorical_features=['marque', 'modele', 'generation', 'finition', 'annee', 'energie', 'batterie', 'boite']):
        self.data = data.copy()
        self.target_features = target_features
        self.categorical_features = categorical_features
        self.missing_rows_index = self.data[self.data[self.target_features].isnull().any(axis=1)].index.to_numpy()

    def impute(self):
        # Filtrer les lignes avec des valeurs manquantes dans les colonnes cibles

        for index, row in self.data[self.data[self.target_features].isnull().any(axis=1)].iterrows():
            # Sélection des observations similaires
            similar_obs = self.data[
                (self.data['marque'] == row['marque']) &
                (self.data['modele'] == row['modele']) &
                (self.data['annee'] == row['annee']) & 
                (self.data['energie'] == row['energie']) & 
                (self.data['boite'] == row['boite']) &
                (self.data['generation'] == row['generation']) 
            ]
            similar_obs1 = similar_obs.loc[similar_obs.index != index]
            if not similar_obs1.empty:
                for feature in self.target_features:
                    self.data.at[index, feature] = similar_obs1.iloc[0][feature]
    
        return self.data
imputer = ImputerCylindrePuissance(data.to_pandas())
imputed_data = imputer.impute()
imputed_data
```